<a href="https://colab.research.google.com/github/KwonDoRyoung/ABRLaboratory/blob/main/0722/3_eval_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import transforms and datasets in TorchVision
import torchvision.transforms as T
import torchvision.datasets as datasets

# load or download MNIST datasets
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=T.ToTensor())

In [ ]:
# import DataLoader in PyTorch
from torch.utils.data import DataLoader

# create data loaders to feed data into our model
batch_size = 64
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# import nn in PyTorch
import torch
import torch.nn as nn

# define MLP network with one hidden layer (original version)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten() # flatten the input tensor as a 1D vector ((28, 28) -> (784))
        self.input_layer = nn.Linear(28*28, 512)
        self.hidden_layer = nn.Linear(512, 256)
        self.output_layer = nn.Linear(256, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        h = self.relu(self.input_layer(x))
        h = self.relu(self.hidden_layer(h))
        y = self.output_layer(h)
        return y

In [ ]:
# import cuda in PyTorch
import torch.cuda as cuda

# check device for training
device = 'cuda' if cuda.is_available() else 'cpu'

# get the MLP model and send it to the device
myMLP = MLP().to(device)

In [ ]:
# load trained weight
import os
import sys

save_path = 'model.pt'

if os.path.exists(save_path):
    myMLP.load_state_dict(torch.load(save_path))
else:
    print(f'There is no checkpoint filename {save_path}')
    sys.exit()

In [ ]:
# test the MLP model
ndata = len(test_dataloader.dataset)
nbatch = len(test_dataloader)
myMLP.eval() # test mode
correct = 0 # initialize

with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        prediction = myMLP(X)
        correct += (prediction.argmax(1) == y).type(torch.float).sum().item() # add up the correct predictions
correct /= ndata # accuracy for all data
print(f"Test accuracy: {(100*correct):>0.1f}%")

Test accuracy: 89.6%
